In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

import cv2

from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
# dimensions of our images.
img_width, img_height = 150, 150
train_data_dir = 'lalit_anuj/train'
validation_data_dir = 'lalit_anuj/valid'
nb_train_samples = 2400
nb_validation_samples = 1000
epochs = 10
batch_size = 16

In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
K.clear_session()

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# this is the augmentation configuration we will use for testing:
# only rescaling
valid_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
model.fit_generator( train_generator, steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs, validation_data=validation_generator, 
                    validation_steps=nb_validation_samples // batch_size)

In [ ]:
test_data_dir = 'lalit_anuj/test'

testing_datagen = ImageDataGenerator(rescale=1. / 255)

testing_generator = testing_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=5,
    class_mode='binary')

In [ ]:
test_imgs,test_labels = next(testing_generator)
plt.imshow(test_imgs[1])

In [ ]:
test_labels

In [ ]:
res = model.predict_generator(testing_generator, steps=1,verbose=1)

In [ ]:
y_pred = [int(i) for i in res]

## Prediction on newly generated Data.

In [ ]:
def takeSelfie():
    cap = cv2.VideoCapture(0)
    if cap.isOpened():
        rest,nImg = cap.read()
        #cv2.imwrite('newImage.jpg',nImg)
        cap.release()
    cv2.destroyAllWindows()
    return nImg

In [ ]:
nImg = takeSelfie()
plt.imshow(nImg)

In [ ]:
def preProcessImage(nImg):
    nImg = cv2.resize(nImg,(150,150))
    imgg = nImg.reshape(1,nImg.shape[0],nImg.shape[1],nImg.shape[2])
    testing_datagenRealWorld = ImageDataGenerator(rescale=1. / 255)
    imgg = testing_datagenRealWorld.flow(imgg,batch_size=1)
    image = next(imgg)
    return (image)
    

In [ ]:
pImg = preProcessImage(nImg) 
res = model.predict(pImg)
res = res.tolist()

In [ ]:
#print(type(res[0][0]))
if int(res[0][0]) == 0 :
    print('You are Anuj')
elif int(res[0][0]) == 1:
    print('You are Lalit')